In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from utils import generate_constrained_random
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

In [3]:
data = pd.read_csv('../../../Dataset/Spambase/spambase.data.txt', header=None)

In [4]:
y = data.pop(57).values
X = data.values
feature_names = data.columns
print X.shape, y.shape

(4601, 57) (4601,)


In [5]:
X_random = generate_constrained_random(X, feature_names, 1000)
print X_random.shape

(1000, 57)


In [12]:
lr = LogisticRegression()
lr.fit(X, y)
print lr.score(X, y)
y_random = lr.predict(X_random.values)

0.93218865464


In [13]:
num_features = X_random.shape[1]
print num_features
y_random = pd.get_dummies(y_random).values

57


In [37]:
tf.reset_default_graph()

with tf.variable_scope("Placeholders") as scope:
    input_email = tf.placeholder(dtype=tf.float32, shape=[None, num_features], name="input_email")
    label = tf.placeholder(dtype=tf.float32, shape=[None, 2], name="labels")
    
num_hidden = 1024
with tf.variable_scope("hidden_layer") as scope:
    w = tf.get_variable(name="weights", shape=[num_features, num_hidden], dtype=tf.float32, initializer=tf.random_normal_initializer())
    b = tf.get_variable(name="biases", shape=[num_hidden], initializer=tf.random_normal_initializer(), dtype=tf.float32)
    act = tf.matmul(input_email, w) + b
    out = tf.nn.relu(act)
    
with tf.variable_scope("output_layer") as scope:
    w = tf.get_variable(name="weights", shape=[num_hidden, 2], dtype=tf.float32, initializer=tf.random_normal_initializer())
    b = tf.get_variable(name="biases", shape=[2], dtype=tf.float32, initializer=tf.random_normal_initializer())
    softmax = tf.nn.softmax(tf.matmul(out, w) + b)
    
with tf.variable_scope("loss") as scope:
    loss = tf.reduce_mean(-tf.reduce_sum(label*tf.log(softmax + 1e-10), reduction_indices=[1]))
    
with tf.variable_scope("optimizer") as scope:
    optimizer = tf.train.AdagradOptimizer(learning_rate=1e-2).minimize(loss)
    
with tf.variable_scope("accuracy") as scope:
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(softmax, 1), tf.arg_max(label, 1)), dtype=tf.float32)) * 100
    
with tf.variable_scope("grads") as scope:
    var = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='hidden_layer')[0]
    grads = tf.gradients(loss, var)

In [38]:
N_EPOCHS = 20
SKIP_STEP = 1
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(1, N_EPOCHS+1):
        
        _, l, accuracy, grad = sess.run([optimizer, loss, acc, grads], feed_dict={input_email:X_random.values, 
                                                                     label:y_random})
        
        if i % SKIP_STEP == 0:
            
            print 'Iteration: {}\n Loss: {}\t Accuracy: {}'.format(i, l, accuracy)
            
    
    y_pred = sess.run([softmax], feed_dict={input_email:X})
    

Iteration: 1
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 2
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 3
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 4
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 5
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 6
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 7
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 8
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 9
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 10
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 11
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 12
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 13
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 14
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 15
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 16
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iteration: 17
 Loss: 3.2005944252	 Accuracy: 86.0999984741
Iterat

In [39]:
print grad[0]

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [40]:
lr = LogisticRegression()
lr.fit(X_random, np.argmax(y_random,1))
print lr.score(X, y)

0.85242338622


In [41]:
np.mean(np.equal(np.argmax(y_pred[0], 1), y))

0.6035644425124973